## Análisis Precipitación Total Parque Nacional Terepaima

**PROYECTO:** SISTEMA PARA EL SEGUIMIENTO DE ECOSISTEMAS VENEZOLANOS \
**AUTOR:** Javier Martinez

In [1]:
import os

print('> Directorio actual: ', os.getcwd())  
os.chdir('../')
print('> Directorio actual: ', os.getcwd()) 

> Directorio actual:  /media/javier/Compartida/doctorado/ssev-analytics/cerro_saroche
> Directorio actual:  /media/javier/Compartida/doctorado/ssev-analytics


In [3]:
import pandas as pd
import pickle

from utils.MONGO import CONEXION
from utils.UTILS import *
from datetime import datetime


In [2]:
park = 'terepaima'

## Creando Coenxión con Mongo DB

In [4]:
# Creando la conexión con MongoDB
db = CONEXION.conexion()
db.list_collection_names()

['estimateMeteorological', 'meteorological', 'estimateSSTNino34', 'SSTNino34']

## Descargando la Información Precipitación

In [5]:
# Realizando consulta
meteorological = db.meteorological.find({"park":park,})

# Generando pandas dataframe
data_pandas = pd.DataFrame([file for file in meteorological])
data_pandas['periodo'] = data_pandas.time.apply(lambda x: datetime.fromordinal(x))
data_pandas['mes_year'] =  data_pandas['periodo'].dt.strftime('%B-%Y')
data_pandas.index = pd.to_datetime(data_pandas.periodo)

data_pandas.head()

,_id,id_point,park,time,elevacion_maxima,elevacion_media,elevacion_mediana,latitud,longitud,ndvi_maxima,ndvi_media,ndvi_mediana,precipitacion_mm,time_actualizacion,periodo,mes_year
periodo,,,,,,,,,,,,,,,,
1970-01-01,6354801a343c9f2921d883bf,1,cerro_saroche,719163,921.0,508.541046,491.0,10.31,-69.83,NaN,NaN,NaN,0.913065,738450,1970-01-01,January-1970
1970-02-01,6354801a343c9f2921d883d3,1,cerro_saroche,719194,921.0,508.541046,491.0,10.31,-69.83,NaN,NaN,NaN,0.081278,738450,1970-02-01,February-1970
1970-03-01,6354801a343c9f2921d883e0,1,cerro_saroche,719222,921.0,508.541046,491.0,10.31,-69.83,NaN,NaN,NaN,0.413783,738450,1970-03-01,March-1970
1970-04-01,6354801a343c9f2921d883ee,1,cerro_saroche,719253,921.0,508.541046,491.0,10.31,-69.83,NaN,NaN,NaN,0.895653,738450,1970-04-01,April-1970
1970-05-01,6354801b343c9f2921d88407,1,cerro_saroche,719283,921.0,508.541046,491.0,10.31,-69.83,NaN,NaN,NaN,2.909450,738450,1970-05-01,May-1970


Consulta de la data

In [6]:
fig_pre = precipitacion_graf( data_pandas,
                    rows=5, 
                    cols=3,
                    park = 'Cerro Saroche'
                    )

fig_pre.write_image(f'./{park}/figures/precipitacion_cerro_saroche.png')
fig_pre.show()

In [7]:
pd_idPoints = data_pandas\
                    .groupby(['id_point', 'latitud', 'longitud','elevacion_media'],as_index=False)\
                    .count()[['id_point', 'latitud', 'longitud','elevacion_media']]

pd_idPoints.head(15)

,id_point,latitud,longitud,elevacion_media
0,1,10.31,-69.83,508.541046
1,2,10.31,-69.73,625.942932
2,3,10.31,-69.63,731.954834
3,4,10.31,-69.53,761.129150
4,5,10.31,-69.43,726.967285
5,6,10.21,-69.83,604.697083
6,7,10.21,-69.73,652.016235
7,8,10.21,-69.63,850.282715
8,9,10.21,-69.53,769.925049
9,10,10.21,-69.43,919.076721


## Resultados de experimentos

In [8]:
DIR = f'./{park}/experiments/narx/precipitacion/'
experiments = [DIR + x for x in os.listdir(DIR)]

In [9]:
# Resultados
pd_summary = pd.concat(list(map(
                                lambda expe: pd.concat([pd.read_csv(expe + '/' +x) for x in os.listdir( expe ) if x.find('summary')!=-1 ]),
                                experiments)))

pd_summary.head()

,epocas,prediction_order,auto_order,exog_order,exog_delay,activation,id_point,n_neurons,capas,training_mse,...,validation_mse,validation_rmse,validation_mae,validation_mape,validation_r,test_mse,test_rmse,test_mae,test_mape,test_r
0,119,24,300,120,3,sigmoid,1,[420],1,0.646958,...,1.549631,2.401356,0.875420,75.907106,-0.089336,1.570334,2.465949,0.870999,70.603053,-0.118638
0,355,24,300,300,0,sigmoid,1,[400],1,0.337411,...,1.534920,2.355979,0.955594,119.693616,-0.068752,1.523241,2.320264,0.936410,113.957562,-0.052550
0,119,12,300,120,3,sigmoid,1,"[420, 210]",2,0.475246,...,1.146938,1.315467,0.891520,148.703450,-0.408240,1.136143,1.290821,0.889697,153.636420,-0.381857
0,120,12,300,12,3,sigmoid,1,"[624, 312]",2,0.281935,...,0.915359,0.837882,0.587512,52.103188,0.103027,0.926219,0.857881,0.567638,48.085447,0.081617
0,94,24,300,120,3,sigmoid,1,"[280, 140]",2,0.719280,...,1.589341,2.526003,0.872345,61.771155,-0.145881,1.626902,2.646810,0.891094,59.207421,-0.200683


Resultados mejores modelos

In [10]:
list_confi = []

for id in pd_summary.sort_values('id_point').id_point.unique():
    # Configuracion
    with open(f'{DIR}id_point_{id}/model_confi.pkl', 'rb') as f:
        model_confi = pickle.load(f)

    list_confi.append( pd.DataFrame(model_confi['metrics'],index=[0]) )

resultados = pd.concat(list_confi)
resultados.to_csv(f'./{park}/data/narx_precipitacion_metricas.csv',index=False)
resultados.head(15)

,epocas,prediction_order,auto_order,exog_order,exog_delay,activation,id_point,n_neurons,capas,training_mse,...,validation_mse,validation_rmse,validation_mae,validation_mape,validation_r,test_mse,test_rmse,test_mae,test_mape,test_r
0,172,12,300,6,3,sigmoid,1,"[306, 204, 102]",3,0.236419,...,0.752806,0.566717,0.550651,66.601822,0.393316,0.705353,0.497523,0.536908,64.706166,0.467389
0,126,12,300,6,3,sigmoid,2,"[204, 136, 68]",3,0.254656,...,0.706064,0.498526,0.513952,53.900047,0.478890,0.677202,0.458602,0.479733,48.838112,0.520622
0,135,12,300,6,3,sigmoid,3,"[204, 136, 68]",3,0.230397,...,0.705857,0.498234,0.459815,35.251702,0.466069,0.785406,0.616862,0.495312,33.771971,0.338943
0,137,12,300,120,3,sigmoid,4,"[168, 126, 84, 42]",4,0.217725,...,0.736733,0.542776,0.513487,49.332610,0.412906,0.719963,0.518347,0.492670,48.642678,0.439330
0,127,12,300,120,3,sigmoid,5,"[168, 126, 84, 42]",4,0.281770,...,0.755652,0.571010,0.595073,62.050576,0.427770,0.755419,0.570659,0.592988,63.827277,0.428122
0,146,12,300,6,3,sigmoid,6,"[306, 204, 102]",3,0.305657,...,0.738563,0.545475,0.533712,53.048594,0.460223,0.744478,0.554248,0.524180,48.820964,0.451542
0,144,12,300,6,3,sigmoid,7,"[306, 204, 102]",3,0.262403,...,0.708569,0.502070,0.443606,36.067884,0.537362,0.714638,0.510708,0.420123,32.839690,0.529403
0,142,12,300,120,3,sigmoid,8,"[168, 126, 84, 42]",4,0.287237,...,0.645995,0.417310,0.499291,50.390485,0.609382,0.612118,0.374689,0.479277,50.824243,0.649277
0,132,12,300,120,3,sigmoid,9,"[210, 140, 70]",3,0.281475,...,0.695732,0.484043,0.500414,44.494674,0.536896,0.677154,0.458538,0.476622,43.729938,0.561298
0,113,12,300,120,3,sigmoid,10,"[168, 126, 84, 42]",4,0.446580,...,0.722940,0.522643,0.524837,43.776897,0.526642,0.717827,0.515275,0.528724,44.880563,0.533315


Pronóstico de los mejores modelos

In [11]:
prediction_order = resultados.prediction_order.min()
data_precipitacion = []

for id in pd_summary.id_point.unique():
    data = pd.read_pickle( f'{DIR}id_point_{id}/predicciones.pkl' ).reset_index(drop=False)

    data['id_point'] = id
    data['park'] = park
    data['periodo'] =  pd.to_datetime(data.periodo)
    #data['periodo'] =  data['index']

    data = pd.merge(data, pd_idPoints, on = ['id_point'], how='left')[['park','periodo','id_point','latitud','longitud','type','precipitacion_mm','prediction_precipitacion_mm','elevacion_media']]

    max_date = data.query("type=='prediction'")[:prediction_order].periodo.max()

    data_precipitacion.append(data[data.periodo<=max_date].query("(type=='training' or type=='test' or type=='prediction')"))

precipitacion_pd = pd.concat(data_precipitacion)
precipitacion_pd.head()

,park,periodo,id_point,latitud,longitud,type,precipitacion_mm,prediction_precipitacion_mm,elevacion_media
0,cerro_saroche,1995-01-01,1,10.31,-69.83,training,0.072351,0.108708,508.541046
1,cerro_saroche,1995-02-01,1,10.31,-69.83,training,0.340843,0.299087,508.541046
2,cerro_saroche,1995-03-01,1,10.31,-69.83,training,2.290730,2.657068,508.541046
3,cerro_saroche,1995-04-01,1,10.31,-69.83,training,1.064486,1.260912,508.541046
4,cerro_saroche,1995-05-01,1,10.31,-69.83,training,1.114330,1.177589,508.541046


## Gráfico NARX precipitación 

In [12]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

rows = 5
cols = 3
fig = make_subplots(rows=rows,
                    cols=cols,
                    subplot_titles='test',
                    horizontal_spacing=0.1,
                    vertical_spacing=0.13)

#------------------
def model_trace(precipitacion_pd, row, col):

    data_figure_ajuste = precipitacion_pd.query("type=='training'")
    data_figure_validacion = precipitacion_pd.query("type=='test'")
    data_figure_pronostico = precipitacion_pd.query("type=='prediction'")

    if data_figure_ajuste.id_point.unique()[0]==1:
        showlegend=True
    else:
        showlegend=False

    marker_line_width=1.5
    marker_size=1
    line_width=2
    fig.add_trace(go.Scatter(x=data_figure_ajuste.periodo, y=data_figure_ajuste['prediction_precipitacion_mm'],
                             mode='lines+markers',name='Pronóstico entrenamiento',
                             marker_symbol='hexagram',
                             marker_line_width=marker_line_width,
                             marker_size=marker_size,
                             marker_line_color='#009DD8',
                             marker_color='#009DD8',
                             line=dict(color='#009DD8', width=line_width),
                             legendgroup='group1',
                             showlegend=showlegend
                            ),     
            row=row, 
            col=col)

    fig.add_trace(go.Scatter(x=data_figure_ajuste.periodo, y=data_figure_ajuste['precipitacion_mm'],
                             mode='lines+markers',name='Precipitación entrenamiento',
                             marker_symbol='x-thin',
                             marker_line_width=marker_line_width,
                             marker_size=marker_size,
                             marker_line_color='#0055D8',
                             marker_color='#0055D8',
                             line=dict(color='#0055D8', width=line_width),
                             legendgroup='group2',
                             showlegend=showlegend
                            ),     
            row=row, 
            col=col)

    fig.add_trace(go.Scatter(x=data_figure_validacion.periodo, y=data_figure_validacion['prediction_precipitacion_mm'],
                        mode='lines+markers',name='Pronóstico validación',                       
                             marker_symbol='hexagram',
                             marker_line_width=marker_line_width,
                             marker_size=marker_size,
                             marker_line_color='#00B6FF',
                             marker_color='#00B6FF', 
                             line=dict(color='#00B6FF', width=line_width),
                             legendgroup='group3',
                             showlegend=showlegend
                            ),       
            row=row, 
            col=col)

    fig.add_trace(go.Scatter(x=data_figure_validacion.periodo, y=data_figure_validacion['precipitacion_mm'],
                        mode='lines+markers',name='Precipitación validación',
                        marker_symbol='square',
                             marker_line_width=marker_line_width,
                             marker_size=marker_size,
                        marker_line_color='#009BD9',
                        marker_color='#009BD9', 
                        line=dict(color='#009BD9', width=line_width),
                        legendgroup='group4',
                        showlegend=showlegend
                            ),
            row=row, 
            col=col)


    fig.add_trace(go.Scatter(x=data_figure_pronostico.periodo, y=data_figure_pronostico['prediction_precipitacion_mm'],
                             text=data_figure_pronostico['prediction_precipitacion_mm'].apply(lambda x: str(round(x,2)) ),
                             textposition="top right",
                             marker_symbol='star',
                             marker_line_width=marker_line_width,
                             marker_size=marker_size,
                             marker_line_color='#29E8FF',
                             marker_color='#29E8FF',
                             mode='lines+markers',name='Pronóstico Precipitación',
                             line=dict(color='#29E8FF', width=line_width,dash='dot'),
                            legendgroup='group5',
                            showlegend=showlegend
                            ),
            row=row, 
            col=col)

    fig.add_vline(x=data_figure_ajuste.periodo.max(), line_width=2, line_dash="dash", line_color="#580606")
    fig.add_vline(x=data_figure_validacion.periodo.max(), line_width=2, line_dash="dash", line_color="#580606")
    
    fig.update_xaxes(tickformat="%Y/%m",
                     showline=True, 
                     linewidth=1, 
                     linecolor='black',
                     gridcolor='#E4E4E4',
                     mirror=True,
                     ticks="outside", 
                     tickwidth=2, 
                     tickcolor='#5C2B05',
                     ticklen=10,
                    range=[data_figure_ajuste.periodo.max() - pd.DateOffset(months=4*12), data_figure_pronostico.periodo.max() + pd.DateOffset(months=3)],
                    title_text='Mes',
                    title_font=dict(size=12)
                    )
    fig.update_yaxes(showline=True,
                     linewidth=1,
                     linecolor='black', 
                     gridcolor='#E4E4E4',
                     mirror=True,
                     ticks="outside", 
                     tickwidth=2, 
                     tickcolor='#5C2B05',
                     ticklen=10,
                     range=[0,10],
                     title_text='Precipitación (mm)',
                     title_font=dict(size=12))

    return fig
#----------------------

In [13]:
rows=5 
cols=3


from plotly.subplots import make_subplots
import plotly.graph_objects as go

column_titles = list(map(lambda x: 'Punto id:' + str(int(x)), precipitacion_pd.sort_values('id_point').id_point.unique().tolist() ))

fig = make_subplots(rows=rows,
                    cols=cols,
                    subplot_titles=column_titles,
                    horizontal_spacing=0.1,
                    vertical_spacing=0.13)


row_order = precipitacion_pd.sort_values('id_point').id_point.unique().reshape(rows,cols).tolist()

for row in range(len(row_order)):

    list_row = row_order[row]
    for col in range(len(list_row)):
        # print(row_order[row][col])

        model_trace(precipitacion_pd.query(f"id_point=={row_order[row][col]}"),
                    row=row+1, col=col+1)
        # print(row)
        # print(col)
    

#------------
# fig['layout']['title']['y']=0.98
# fig['layout']['margin']['t']=100

#------------
annotations_title = fig['layout']['annotations']

y_loc=7.5
yshift=4
annotations = []

for id_ in precipitacion_pd.sort_values('id_point').id_point.unique().tolist():
    
    xref='x{id_}'.format(id_=str(int(id_)))
    yref='y{id_}'.format(id_=str(int(id_)))
    
    titulo_annotations =  list(filter(lambda x: x.text=='Punto id:{id_}'.format(id_=int(id_)),
                                  annotations_title))[0]

    annotations= annotations+[ dict(font=dict(size=16),
                                    showarrow=titulo_annotations['showarrow'],
                                    text=titulo_annotations['text'],
                                    x=titulo_annotations['x'],
                                    xanchor=titulo_annotations['xanchor'],
                                    xref='paper',#xref,
                                    y=titulo_annotations['y'],
                                    yanchor=titulo_annotations['yanchor'],
                                    yref='paper',#yref
                                    ),
                                dict(
                                    x=precipitacion_pd.query("type=='training'").periodo.max() - pd.DateOffset(months=12*2),
                                    y=y_loc, # annotation point
                                    xref=xref, 
                                    yref=yref, 
                                    text='Entrenamiento',
                                    showarrow=False,
                                    yshift=yshift,
                                    font=dict(size=9)
                                  ),
                                dict(
                                    x=precipitacion_pd.query("type=='test'").periodo.max() - pd.DateOffset(months=6),
                                    y=y_loc, # annotation point
                                    xref=xref, 
                                    yref=yref, 
                                    text='Validación',
                                    showarrow=False,
                                    yshift=yshift,
                                    font=dict(size=9)
                                  ),
                                dict(
                                    x=precipitacion_pd.query("type=='prediction'").periodo.max() - pd.DateOffset(months=6),
                                    y=y_loc, # annotation point
                                    xref=xref, 
                                    yref=yref, 
                                    text='Pronóstico',
                                    showarrow=False,
                                    yshift=yshift,
                                    font=dict(size=9)
                                  )
                                    ]

fig['layout']['annotations'] = annotations

        
#------------       
fig.update_xaxes(tickformat="%Y/%m",showline=True, linewidth=1, linecolor='black', gridcolor='#E4E4E4',mirror=True,
                 ticks="outside", tickwidth=2, tickcolor='#5C2B05', ticklen=10)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', gridcolor='#E4E4E4',mirror=True,
                 ticks="outside", tickwidth=2, tickcolor='#5C2B05', ticklen=10)   

#------------
fig.update_layout( height=800,
                   width=1500,
                   #showlegend = True,
                   font = dict(size = 12),
                   template='plotly_white',
                   title_text="""Precipitación total Cerro Saroche
                        <br><sup>Pronóstico para el periodo {date_init} al {date_fin}</sup>
                        """.format(date_init=str(precipitacion_pd.query("type=='prediction'").periodo.min().strftime('%Y/%m')),
                                   date_fin=str(precipitacion_pd.query("type=='prediction'").periodo.max().strftime('%Y/%m'))),
                  legend_title_text='Serie',
                  legend_title = dict( font = dict(size = 14)),
                   uniformtext_minsize=8,
                   uniformtext_mode='hide',
                   legend = dict(
                                # #orientation="h",
                                # yanchor="bottom",
                                # y=1.05,
                                # xanchor="right",
                                # x=1,
                                # font_size=14
                            )
                 )

fig.write_image(f'./{park}/figures/narx_precipitacion.png')
fig.show()

## Data para variación espacio-temporal de la precipitación

Predicciones

In [14]:
columns = ['park',	'periodo', 'year', 'month',	'id_point',	'latitud',	'longitud',	'type',	'precipitacion_mm','prediction_precipitacion_mm', 'elevacion_media']

precipitacion_pd['year'] = precipitacion_pd['periodo'].dt.strftime("%Y").astype(int)
precipitacion_pd['month'] = precipitacion_pd['periodo'].dt.strftime("%m").astype(int)

precipitacion_pd[columns].to_pickle(f'./{park}/data/narx_precipitacion.pkl')